GREATSCHOOL.ORG SCRAPER

Initial Steps in the Terminal:
1. Create a new directory for the project and position ourselves there
mkdir GreatSchoolScraper
cd GreatSchoolScraper
2. Update pip just in case it has an old version
pip install --upgrade pip
3. Create a virtual environment and activate it
python3 -m venv ./
source bin/activate
4. Install needed libraries
pip install pandas selenium webdriver-manager

In [ ]:
# Do not run. Finding schools in ListMap View
schools_selector = driver.find_elements(By.CSS_SELECTOR, '.school-list li.unsaved')

schools = []
for school in schools_selector:
    school_url = school.find_element(By.CLASS_NAME, 'name').get_attribute('href')
    school_name = school.find_element(By.CLASS_NAME, 'name').text
    try:
        school_rating = school.find_element(By.CLASS_NAME, 'circle-rating--small').text
    except:
        school_rating = 'Unrated'
    school_address = school.find_element(By.CLASS_NAME, 'address').text
    
    columns = school.find_elements(By.CSS_SELECTOR, 'td')

    schools.append({'school_name': school_name,
                    'school rating': school_rating,
                    'school_address': school_address,
                    'school_url': school_url)


In [1]:
!python3 -m pip install --upgrade pip

In [18]:
# Import libraries
#import pandas as pd
import time
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import os
import logging
from selenium.webdriver.support.ui import Select

In [ ]:
import token
from twilio.rest import Client
# Set environment variables for your credentials
# Read more at http://twil.io/secure
account_sid = "AC76e55aab103ecc85c59dcb3b7189444e"
auth_token = '6003a2aa36d6aa34502e933033a4b386' #os.environ["TWILIO_AUTH_TOKEN"]


In [84]:
## Setup chrome options
chrome_options = Options()
chrome_options.add_argument("--headless") # Ensure GUI is off
chrome_options.add_argument("--no-sandbox")

# Silent download of drivers
logging.getLogger('WDM').setLevel(logging.NOTSET)
os.environ['WDM_LOG'] = 'False'

# Creating a driver objet to manage browser 
driver = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)

# Defining the url we want to scrape
url_pages = {#"Lakewood":"https://www.greatschools.org/california/lakewood/schools/",
             #"Cerritos":"https://www.greatschools.org/california/cerritos/schools/",
             "Travel Lodge": "https://reservations.ahlsmsworld.com/Yosemite/Plan-Your-Trip/Accommodations",} #?gradeLevels%5B%5D=e&gradeLevels%5B%5D=m&st%5B%5D=public_charter&st%5B%5D=public&st%5B%5D=charter
             #"Bellflower":"https://www.greatschools.org/california/bellflower/schools/",
             #"Norwalk":"https://www.greatschools.org/california/norwalk/schools/",
             #"Downey":"https://www.greatschools.org/california/downey/schools/",
             #"Whittier":"https://www.greatschools.org/california/whittier/schools/",
             #"LosAlamitos":"https://www.greatschools.org/california/los-alamitos/schools/",
             #"Cypress":"https://www.greatschools.org/california/cypress/schools/",
             #"SantaFeSprings":"https://www.greatschools.org/california/santa-fe-springs/schools/"}

#every_school_df     = pd.DataFrame()
#every_test_score_df = pd.DataFrame()
#every_demo_info_df  = pd.DataFrame()

# Get data from all the locations of interest

for page in url_pages:    
    # Ask the driver to direct us to the url
    driver.get(url_pages[page])
    print("Processing" , url_pages[page] )
    # Get table view
    time.sleep(2)
    
    select = Select(driver.find_element(By.ID,'box-widget_ProductSelection'))
    # select by visible text
    select.select_by_visible_text('Yosemite Valley Lodge')
    time.sleep(1)

    select = Select(driver.find_element(By.ID,'box-widget_UnitCount'))
    
    select.select_by_visible_text('3')

    select = Select(driver.find_element(By.ID,'box-widget_MultiUnitCount_Children[0]'))
    select.select_by_visible_text('1')
    select = Select(driver.find_element(By.ID,'box-widget_MultiUnitCount_Children[1]'))
    select.select_by_visible_text('1')
    select = Select(driver.find_element(By.ID,'box-widget_MultiUnitCount_Children[2]'))
    select.select_by_visible_text('1')
    
    #driver.find_element(By.CLASS_NAME,"input-group-addon").click();
    #time.sleep(3)
    driver.find_element(By.ID, "box-widget_ArrivalDate").click()

    select = Select(driver.find_element(By.CLASS_NAME,'ui-datepicker-month'))
    select.select_by_visible_text('Jul')
    #time.sleep(2)
    #driver.implicitly_wait(10)
    #ActionChains(driver).move_to_element(button).click(button).perform()

    days_selector = driver.find_elements(By.CSS_SELECTOR, 'tr')
    
    avail = driver.find_elements(By.CSS_SELECTOR, '.ui-datepickerAvail-significant:not(.ui-datepicker-unselectable), .ui-datepickerAvail-limited:not(.ui-datepicker-unselectable)')
    print(len(avail))
    days = []

    from selenium.common.exceptions import StaleElementReferenceException

    #
    #    passwordElement.send_keys("password")
    #except StaleElementReferenceException as Exception:
    try:
        for day in avail:
            if day.text != '':
                days.append(int(day.text))
    except StaleElementReferenceException as Exception:
         print('no hay dias disponibles')      
        #    columns = driver.find_elements(By.CSS_SELECTOR, 'td')
        #    print(columns[0].class)
        #    #for i,j in enumerate(columns):
        #    #    print(columns[i].text)
    finally:
        counter = 1
        for i,day in enumerate(days):
            if i == 0: 
                continue
            if days[i] == days[i-1] + 1:
                counter += 1
            else: 
                counter = 1

            if counter == 4:
                break

        if counter > 4:
            client = Client(account_sid, auth_token)
        # message = client.messages.create( body="Hay disponibilidad!",from_="+18882985392",to="+15629556375")
    

   
    

/var/folders/hs/lpx8vtgn44v54ln3t8dg97vr0000gn/T/ipykernel_24967/3729761519.py:11: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)


Processing https://reservations.ahlsmsworld.com/Yosemite/Plan-Your-Trip/Accommodations
33
no hay dias disponibles


In [19]:
def availabity(driver):    
    """ Get schools from the selected page in Table View. 
    
    Arguments:
    driver -- object containing the driver handle
    
    Output: 
    schools -- list with all the schools from the table in the selected page
    """

    schools_selector = driver.find_elements(By.CSS_SELECTOR, '.school-table tr')

    # For every school in the list, get interesting attributes. It skips the header and starts with the first row of data
    schools = []
    for school in schools_selector[1:]:
        school_url = school.find_element(By.CLASS_NAME, 'name').get_attribute('href')
        school_name = school.find_element(By.CLASS_NAME, 'name').text
        school_address = school.find_element(By.CLASS_NAME, 'address').text
        # Sometimes the rating doesn't exist
        try:
            school_rating = school.find_element(By.CLASS_NAME, 'circle-rating--small').text
        except:
            school_rating = 'Unrated'

        # The rest of the attributes are don't have a class name, so we create a list with the data from each column
        # and access it directly
        columns = school.find_elements(By.CSS_SELECTOR, 'td')

        schools.append({'school_name': school_name,
                        'school_rating': school_rating,
                        'school_address': school_address,
                        'school_url': school_url,
                        'school_type': columns[1].text,
                        'school_grades': columns[2].text,
                        'school_total_students': columns[3].text,
                        'school_teacher_ratio': columns[4].text,
                        'school_district': columns[6].text})
        print("School ", school_name)

    return schools

In [20]:
def extra_info(schools):
    """ Get available additional information (Academic and Demographic) from each school in the dataframe using its url. 
    
    Arguments:
    schools -- pandas dataframe containing basic school information,
    
    Output: 
    test_scores -- list with subjects and test scores. 
    demo_info -- list with races and percentages.
    """

    test_scores = []
    demo_info = []
    
    for i in schools.index:     #school in schools:
        
        # Go to school page
        url = schools['school_url'][i]
        driver.get(url)
        time.sleep(3)
        # Get academic info if available
        try:
            academics = driver.find_element(By.ID, 'Test_scores')
            subjects = academics.find_elements(By.CLASS_NAME, value = 'bar-graph-unified')
        except:
            continue

        for subject in subjects:
            subject_name = subject.find_element(By.CLASS_NAME, 'bar-graph-label').text
            subject_score = subject.find_element(By.CLASS_NAME, 'percentage').text
            test_scores.append({'school_name': schools['school_name'][i],
                                'subject_name': subject_name,
                                'subject_score': subject_score})
        
        # Get Demographic Info if available
        try:
            demographic = driver.find_element(By.ID, 'Students')
            races = demographic.find_elements(By.CLASS_NAME, value = 'bar-graph-unified')
        except:
            continue

        for race in races:
            race_name = race.find_element(By.CLASS_NAME, 'bar-graph-label').text
            race_percentage = race.find_element(By.CLASS_NAME, 'percentage').text
            demo_info.append({'school_name': schools['school_name'][i],
                              'race': race_name,
                              'race_score': race_percentage})
        
    return test_scores, demo_info

In [22]:
## Setup chrome options
chrome_options = Options()
chrome_options.add_argument("--headless") # Ensure GUI is off
chrome_options.add_argument("--no-sandbox")

# Silent download of drivers
logging.getLogger('WDM').setLevel(logging.NOTSET)
os.environ['WDM_LOG'] = 'False'

# Creating a driver objet to manage browser 
driver = webdriver.Chrome(ChromeDriverManager().install())

# Defining the url we want to scrape
url_pages = {#"Lakewood":"https://www.greatschools.org/california/lakewood/schools/",
             #"Cerritos":"https://www.greatschools.org/california/cerritos/schools/",
             "https://reservations.ahlsmsworld.com/Yosemite/Plan-Your-Trip/Accommodations",} #?gradeLevels%5B%5D=e&gradeLevels%5B%5D=m&st%5B%5D=public_charter&st%5B%5D=public&st%5B%5D=charter
             #"Bellflower":"https://www.greatschools.org/california/bellflower/schools/",
             #"Norwalk":"https://www.greatschools.org/california/norwalk/schools/",
             #"Downey":"https://www.greatschools.org/california/downey/schools/",
             #"Whittier":"https://www.greatschools.org/california/whittier/schools/",
             #"LosAlamitos":"https://www.greatschools.org/california/los-alamitos/schools/",
             #"Cypress":"https://www.greatschools.org/california/cypress/schools/",
             #"SantaFeSprings":"https://www.greatschools.org/california/santa-fe-springs/schools/"}

every_school_df     = pd.DataFrame()
every_test_score_df = pd.DataFrame()
every_demo_info_df  = pd.DataFrame()
# Get data from all the locations of interest
for page in url_pages:    
    # Ask the driver to direct us to the url
    driver.get(url_pages[page])
    print("Processing" , url_pages[page] )
    # Get table view
    time.sleep(3)
    driver.find_element(By.XPATH, '//span[text()="Table view"]').click()

    # Iterate over all the pages of the url to get every school in the results 
    all_schools_df = pd.DataFrame()
    while True: 
        # Wait until the page is fully loaded
        time.sleep(6)
        # Scroll down to the end of the page so the paginator is available
        driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
        # Get schools from that page
        each_page_schools = get_schools(driver)
        each_page_schools_df = pd.DataFrame(each_page_schools)
        # Concatenates all the pages from the city
        all_schools_df = pd.concat([all_schools_df,each_page_schools_df], ignore_index=True)
        # Verify if next page selector is available
        try:
            next_button_icon = driver.find_element(By.CSS_SELECTOR, '.pagination-container .icon-chevron-right:not(.rotate-text-180)')
            next_button = next_button_icon.find_element(By.XPATH, "../..")
            if next_button.get_attribute('href') is None:
                break
            else:
                next_button.click()  
        except:   
            break

    # Get academic and demographic information of the schools in the list
    test_scores, demo_info = extra_info(all_schools_df)
    
    test_scores_df = pd.DataFrame(test_scores)
    demo_info_df = pd.DataFrame(demo_info)
    
    all_schools_df.to_csv("data/SchoolsOf" + page + ".csv")
    test_scores_df.to_csv("data/TestScoresOf" + page + ".csv")
    demo_info_df.to_csv("data/DemoInfoOf" + page + ".csv")

    every_school_df     = pd.concat([every_school_df,all_schools_df], ignore_index=True)
    every_test_score_df = pd.concat([every_test_score_df,test_scores_df], ignore_index=True)
    every_demo_info_df  = pd.concat([every_demo_info_df,demo_info_df], ignore_index=True)

[WDM] - ====== WebDriver manager ======


2023-02-01 11:26:27,461 INFO ====== WebDriver manager ======


[WDM] - Current google-chrome version is 109.0.5414


2023-02-01 11:26:27,559 INFO Current google-chrome version is 109.0.5414


[WDM] - Get LATEST chromedriver version for 109.0.5414 google-chrome


2023-02-01 11:26:27,562 INFO Get LATEST chromedriver version for 109.0.5414 google-chrome


[WDM] - Driver [/Users/Patricia/.wdm/drivers/chromedriver/mac64/109.0.5414.74/chromedriver] found in cache


2023-02-01 11:26:27,636 INFO Driver [/Users/Patricia/.wdm/drivers/chromedriver/mac64/109.0.5414.74/chromedriver] found in cache


/var/folders/hs/lpx8vtgn44v54ln3t8dg97vr0000gn/T/ipykernel_3983/4204228606.py:11: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


Processing https://www.greatschools.org/california/long-beach/schools/
School  Prisk Elementary School
School  Eunice Sato Academy Of Math & Science
School  Gant Elementary School
School  Henry K-8
School  Kettering Elementary School
School  Los Cerritos Elementary School
School  Naples Elementary School
School  Newcomb Academy
School  Fremont Elementary School
School  Tincher Preparatory School
School  IVA - Intellectual Virtues Academy Middle School
School  Ernest S. McBride Sr. High School
School  Helen Keller Middle
School  Burcham Elementary School
School  Carver Elementary School
School  Cubberley K-8
School  Emerson Parkside Academy
School  Longfellow Elementary School
School  Lowell Elementary School
School  Marshall Academy Of The Arts
School  Rogers Middle School
School  Addams Elementary School
School  Bancroft Middle School
School  Birney Elementary School
School  Bixby Elementary School
School  Franklin Classical Middle School
School  Garfield Elementary School
School  Hug

In [16]:

all_schools_df

,school_name,school rating,school_address,school_url,school_type,school_grades,school_total_students,school_teacher_ratio,school_district
0,Cleveland Elementary School,9\n/10,"4760 Hackett Avenue, Lakewood, CA, 90713",https://www.greatschools.org/california/lakewo...,Public district,K-5,469,21:1,Long Beach Unified School District
1,Odyssey Stem Academy,8\n/10,"3701 Michelson Street, Lakewood, CA, 90712",https://www.greatschools.org/california/lakewo...,Public district,9-12,136,N/A,Paramount Unified School District
2,Stephen Foster Elementary School,7\n/10,"5223 Bigelow Street, Lakewood, CA, 90712",https://www.greatschools.org/california/lakewo...,Public district,K-6,688,30:1,Bellflower Unified School District
3,Gompers K-8,7\n/10,"5206 Briercrest Avenue, Lakewood, CA, 90713",https://www.greatschools.org/california/lakewo...,Public district,K-8,700,24:1,Long Beach Unified School District
4,Holmes Elementary School,7\n/10,"5020 Barlin Avenue, Lakewood, CA, 90712",https://www.greatschools.org/california/lakewo...,Public district,K-5,383,20:1,Long Beach Unified School District
5,Macarthur Elementary School,7\n/10,"6011 Centralia Street, Lakewood, CA, 90713",https://www.greatschools.org/california/lakewo...,Public district,K-5,374,25:1,Long Beach Unified School District
6,Madison Elementary School,7\n/10,"2801 Bomberry Street, Lakewood, CA, 90712",https://www.greatschools.org/california/lakewo...,Public district,K-5,357,22:1,Long Beach Unified School District
7,Intensive Learning Center,6\n/10,"4718 Michelson Street, Lakewood, CA, 90712",https://www.greatschools.org/california/lakewo...,Public district,K-6,761,25:1,Bellflower Unified School District
8,Palms Elementary School,6\n/10,"12445 207th Street, Lakewood, CA, 90715",https://www.greatschools.org/california/lakewo...,Public district,K-6,657,27:1,Abc Unified School District
9,Riley Elementary School,6\n/10,"3319 Sandwood Street, Lakewood, CA, 90712",https://www.greatschools.org/california/lakewo...,Public district,K-5,383,21:1,Long Beach Unified School District


In [105]:
demo_info_df

,school_name,race,race_score
0,Cleveland Elementary School,Hispanic,41%
1,Cleveland Elementary School,White,34%
2,Cleveland Elementary School,Black,6%
3,Cleveland Elementary School,Two or more races,6%
4,Cleveland Elementary School,Filipino,5%
...,...,...,...
154,Hoover Middle School,Unspecified,2%
155,Buena Vista High School,Hispanic,84%
156,Buena Vista High School,Black,12%
157,Buena Vista High School,White,2%


In [106]:
test_scores_df.groupby('school_name').count()


,subject_name,subject_score
school_name,,
Aloha Elementary School,3,3
Artesia High School,3,3
Buena Vista High School,3,3
Cleveland Elementary School,3,3
Craig Williams Elementary School,3,3
Esther Lindstrom Elementary School,3,3
Gompers K-8,3,3
Holmes Elementary School,3,3
Hoover Middle School,3,3


In [122]:
url_pages = {"Lakewood"       : "https://www.greatschools.org/california/lakewood/schools/",
             "Cerritos"       : "https://www.greatschools.org/california/cerritos/schools/",
             "LongBeach"      : "https://www.greatschools.org/california/long-beach/schools/?gradeLevels%5B%5D=e&gradeLevels%5B%5D=m&st%5B%5D=public_charter&st%5B%5D=public&st%5B%5D=charter",
             "Bellflower"     : "https://www.greatschools.org/california/bellflower/schools/",
             "Norwalk"        : "https://www.greatschools.org/california/norwalk/schools/",
             "Downey"         : "https://www.greatschools.org/california/downey/schools/",
             "Whittier"       : "https://www.greatschools.org/california/whittier/schools/",
             "LosAlamitos"    : "https://www.greatschools.org/california/los-alamitos/schools/",
             "Cypress"        : "https://www.greatschools.org/california/cypress/schools/",
             "SantaFeSprings" : "https://www.greatschools.org/california/santa-fe-springs/schools/"}
            

In [125]:
for page in url_pages:
    print(page)

Lakewood
Cerritos
LongBeach
Bellflower
Norwalk
Downey
Whittier
LosAlamitos
Cypress
SantaFeSprings
